In [ ]:
import glob
import tqdm
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
filenames=glob.glob(r"C:/Users/dlara/Downloads\MODELO 3D SANTIAGO CENTRO/*/[a-f]*.obj")

In [ ]:
def map_d(x, in_min, in_max, out_min, out_max):
    """Genera una interpolacion para pasar de un rango a otro"""
    return (x - in_min) * (out_max - out_min) / (in_max - in_min) + out_min

In [ ]:
X=[]
Y=[]
Z=[]
res=[]
for file in tqdm.tqdm(filenames):
    with open(file,"r") as f:
        lines=f.readlines()
    for line in lines:
        if line[:2]=="v ":
            res.append(line.split()+list([file]))

In [ ]:
len(filenames)

In [ ]:
import pandas as pd

In [ ]:
df_coords=pd.DataFrame(res)
df_coords[3]=df_coords[3].astype("float64")
df_coords[1]=df_coords[1].astype("float64")
df_coords[2]=df_coords[2].astype("float64")
df_coords=df_coords.sort_values(by=[3]).reset_index(drop=True)

In [ ]:
df_coords=df_coords.drop_duplicates(subset=[1,2,3]).reset_index(drop=True)
df_coords=df_coords.astype({1:"int",2:int,3:int})
df_coords

In [ ]:
max_coord=np.max(df_coords[3])
min_coord= np.min(df_coords[3])
max_coord, min_coord

In [ ]:
new_image_width = int(np.max(df_coords[1]))+1
new_image_height = int(np.max(df_coords[2]))+1
new_image_height,new_image_width

In [ ]:
df_coords["Z"].mean()

In [ ]:
color = (1601.498848753116)
result = np.full((new_image_height,new_image_width), color, dtype="float32")
# result=result[:,:,0]


In [ ]:
resultbhhblj

In [ ]:
import geopandas as gpd
from shapely.geometry import Point
punto_max=gpd.GeoSeries(Point([-99.772385123212771, 19.792835251567379]),crs=6364)
punto_max.crs

In [ ]:
print(punto_max.to_crs(3857))

In [ ]:
df_coords["Z"]=df_coords[3].apply(lambda x: map_d(x,min_coord,max_coord,2508.7909335236886,2728.701065688485))
df_coords

In [ ]:
import tqdm
for i in tqdm.tqdm(range(len(df_coords))):
    result[df_coords.loc[i,2],df_coords.loc[i,1]]=df_coords.loc[i,3]
    

In [ ]:
result.max()
# result.min()

In [ ]:
with open("E:/aeromodelismo/MODELO_3D_SANTIAGO_NOROESTE/doc.kml","r") as kml:
    li=kml.readlines()
lineas_utiles=[]
for l in li:
    l=l.replace("\n","")
    lineas_utiles.append(l)
    if l.find("href")>=0:
        break
lineas_utiles

In [ ]:
def map_d(x, in_min, in_max, out_min, out_max):
    """Genera una interpolacion para pasar de un rango a otro"""
    return (x - in_min) * (out_max - out_min) / (in_max - in_min) + out_min


In [ ]:
metrosxpixel longitud de array/ diferencia entre west y east en crs:3857

In [ ]:
# Cambié valores latlon , basandome en kml, al crs 3857 para sacar la diferencia entre west y east
# Para la longitud es max lenght
(1257)/(-11105244.964 - (-11106611.107))
# (1257)/(-99.76011285106506 - (-99.772385123212771))


In [ ]:
result

In [ ]:
import rasterio
from affine import Affine
##extremo superior izquierdo

longitud=-11106611.107
latitud=2248505.500
metrosxpixel=0.9201086562673154
crs=3857
matriz=np.flip(result, 0)


transform=Affine(metrosxpixel,0,longitud,0,-metrosxpixel,latitud) ## 0 representa las componetes de direccion de la coordenada
out_meta=({"driver": "GTiff",
            "height": matriz.shape[0],
            "width": matriz.shape[1],
            "count":1,
            "dtype":np.float32,
            "crs":crs,
          'transform':transform})
with rasterio.open(r"D:\blac1k3.tif",'w', **out_meta) as dest_file:
    dest_file.write(matriz, 1)